In [ ]:
import cv2 
import glob
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
# Import images to check

image1 = cv2.imread('images/1985.jpg')
image2 = cv2.imread('images/1995.jpg')
image3 = cv2.imread('images/2005.jpg')
image4 = cv2.imread('images/2015.jpg')

fig = plt.figure(figsize=[20, 10])

ax = fig.add_subplot(2, 2, 1)
ax.set_title('1985')
plt.imshow(image1[:, :, ::-1])

ax = fig.add_subplot(2, 2, 2)
ax.set_title('1995')
plt.imshow(image2[:, :, ::-1])

ax = fig.add_subplot(2, 2, 3)
ax.set_title('2005')
plt.imshow(image3[:, :, ::-1])

ax = fig.add_subplot(2, 2, 4)
ax.set_title('2015')
plt.imshow(image4[:, :, ::-1])

In [ ]:
b1, g1, r1 = cv2.split(image1)
b4, g4, r4 = cv2.split(image4)

# Show the channels.
plt.figure(figsize = [20, 15])
plt.subplot(331); plt.imshow(image1[:, :, ::-1]); plt.title('Original 1985')
plt.subplot(332); plt.imshow(image4[:, :, ::-1]); plt.title('Original 2015')

plt.subplot(334); plt.imshow(r1); plt.title('Red Channel - 1985')
plt.subplot(335); plt.imshow(g1); plt.title('Green Channel - 1985')
plt.subplot(336); plt.imshow(b1); plt.title('Blue Channel - 1985')

plt.subplot(337); plt.imshow(r4); plt.title('Red Channel - 2015')
plt.subplot(338); plt.imshow(g4); plt.title('Green Channel - 2015')
plt.subplot(339); plt.imshow(b4); plt.title('Blue Channel - 2015')

In [ ]:
#Plotting linear and logarithmic histograms

def img_histogam_bgr(img, title = '', yscale = ''):
    hist_b = cv2.calcHist(img, [0], None, [256], [0,255])
    hist_g = cv2.calcHist(img, [1], None, [256], [0,255])
    hist_r = cv2.calcHist(img, [2], None, [256], [0,255])
       
    # Plot the histograms for each channel.
    fig = plt.figure(figsize = [20, 5])
    fig.suptitle(title)
    
    ax = fig.add_subplot(1, 3, 1)
    ax.set_yscale(yscale)
    plt.plot(hist_b, color = 'b', label = 'Blue')
    ax.grid()
    ax.legend()
    
    ax = fig.add_subplot(1, 3, 2)
    ax.set_yscale(yscale)
    plt.plot(hist_g, color = 'g', label = 'Green')
    ax.grid()
    ax.legend()
    
    ax = fig.add_subplot(1, 3, 3)
    ax.set_yscale(yscale)
    plt.plot(hist_r, color = 'r', label = 'Red')
    ax.grid()
    ax.legend()
    
    plt.show()




In [ ]:
# draw histogram wih linear scale
img_histogam_bgr(image1, '1985', 'linear' )
img_histogam_bgr(image2, '1995', 'linear' )
img_histogam_bgr(image3, '2005', 'linear' )
img_histogam_bgr(image4, '2015', 'linear' )



In [ ]:
# drawing it with log scale 
img_histogam_bgr(image1, '1985', 'log' )
img_histogam_bgr(image2, '1995', 'log' )
img_histogam_bgr(image3, '2005', 'log' )
img_histogam_bgr(image4, '2015', 'log' )

In [ ]:
# lets create mask for green color with color segmentation 
# so the lower bound we check in linear/log yscale and found that most of green pixels are in range from 50 to 100 for in BGR
# we select green 50 for lower bound and 100 for higher.

## so actually, inrange will take lower bound and higher bound values and make it white from l to h for all 3 colors. and at the end it inverse.


def detect_green_bgr(img):

    lower_BGR_values = np.array([0, 50, 0], dtype = 'uint8')
    upper_BGR_values = np.array([255,100,255], dtype= 'uint8')

    # Create a mask using the lower and upper range.
    mask_BGR = cv2.inRange(img, lower_BGR_values, upper_BGR_values) 

    return mask_BGR

In [ ]:
def detect_green_HSV(img):
    """Detect and return a mask for the green area of an image using HSV segmentation."""
    HSV_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    lower_HSV_values = np.array([30, 0, 0], dtype = 'uint8')
    upper_HSV_values = np.array([86, 255, 120], dtype = 'uint8')
    
    mask_HSV = cv2.inRange(HSV_image, lower_HSV_values, upper_HSV_values) 
    
    return mask_HSV

In [ ]:
# Drawing green color segmented

segmented_green = detect_green_bgr(image1)

fig = plt.figure(figsize = (20, 10))

ax = fig.add_subplot(1, 2, 1)
plt.imshow(image1[:, :, ::-1])
ax.set_title('Original')

ax = fig.add_subplot(1, 2, 2)
plt.imshow(segmented_green, cmap = 'gray')
ax.set_title('Color Segmented in Green Channel')

plt.show()

In [ ]:
def percent_forest(gray_img):
    """Return the percentage of the image detected to be forested."""
    c = cv2.countNonZero(gray_img)

    # Finding number of pixels in image to find percentage.
    t = gray_img.shape[0] * gray_img.shape[1]

    # Rounding off to 2 decimal place.
    return round((c / t) * 100, 2)

In [ ]:
segmented_green = detect_green_bgr(image1)
green_ratio = percent_forest(segmented_green)

fig = plt.figure(figsize = (20, 20))

ax = fig.add_subplot(1, 2, 1)
plt.imshow(image1[:, :, ::-1])
ax.set_title('Original')

ax = fig.add_subplot(1, 2, 2)
plt.imshow(segmented_green, cmap = 'gray')
ax.set_title('Color Segmented in Green Channel: ' + str(green_ratio) + '%')

plt.show()

In [ ]:
image_files = glob.glob("images/*.jpg")
image_files.sort()
green_ratio_bgr_annual = []
#green_ratio_hsv_annual = []

for image_file in image_files: 
    image = cv2.imread(image_file)
    
    #segmented_green_hsv = detect_green_HSV(image)
    #green_ratio_hsv = percent_forest(segmented_green_hsv)
    #green_ratio_hsv_annual.append(green_ratio_hsv)
    
    segmented_green_bgr = detect_green_bgr(image)
    green_ratio_bgr = percent_forest(segmented_green_bgr)
    green_ratio_bgr_annual.append(green_ratio_bgr)

    fig = plt.figure(figsize = (20, 10))
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(image[:, :, ::-1])
    ax.set_title('Original - ' + image_file)
    
    #ax = fig.add_subplot(1, 3, 2)
    #plt.imshow(segmented_green_hsv, cmap = 'gray')
    #ax.set_title('Color Segmented using HSV Color Space: ' + str(green_ratio_hsv) + '%')
    
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(segmented_green_bgr, cmap = 'gray')
    ax.set_title('Color Segmented using BGR Color Space: ' + str(green_ratio_bgr) + '%')
    
    plt.show()

In [ ]:
## plotting graphs for percentages

years = np.linspace(1984,2020,37,dtype='int64')


plt.plot(years, green_ratio_bgr_annual, color='tab:blue', marker='.', linestyle='--'); 
plt.ylabel('Percentage of Froest %')
plt.xlabel('Years')
plt.title('Deforestation Analysis of Multan - (1984-2020)')
plt.grid()
#plt.plot(years, green_ratio_hsv_annual, 'g')
plt.show() 